# odinson
Retrieve original Marvel Comics editions for stories published in old brazilian comics

##### 2021.11.14

---

## Souces

[Guia dos Quadrinhos](http://www.guiadosquadrinhos.com/)
- [Heróis da TV](http://www.guiadosquadrinhos.com/edicao/herois-da-tv-2-serie-n-1/htv0302/6274)
- [Superaventuras Marvel](http://www.guiadosquadrinhos.com/edicao/superaventuras-marvel-n-1/sam0301/7927)
- [Almanaque Premiere Marvel](http://www.guiadosquadrinhos.com/edicao/almanaque-premiere-marvel-n-1/apm02001/18611)

---


## 1. Develop retrieving:
- edition
    - id="nome_titulo_lb"
- ~cover image? link?~
    - id="ampliar_capa"
- story (portuguese)
    - all: id="texto_pag_detalhe"
- main character
- original publication
- year
- original title

In [ ]:
urlroot = "http://www.guiadosquadrinhos.com/"
# urledition = "edicao/herois-da-tv-2-serie-n-1/htv0302/6274"
urledition = "edicao/superaventuras-marvel-n-1/sam0301/7927"

In [ ]:
urlroot + urledition

### 1.1 dev scrape main #1 page

In [ ]:
# dev scraping
import requests
from bs4 import BeautifulSoup

URL = urlroot + urledition
page = requests.get(URL)

thesoup = BeautifulSoup(page.content, "html.parser")

#### 1.1.1 Edition

In [ ]:
# Edition
def edition(asoup):
    ed = asoup.find(id="nome_titulo_lb")
    edl = ed.contents
    edl = [f"{i}".strip() for i in edl]
    edl = [i.split(">")[1] if ">" in i else i for i in edl]
    edl = [i.split("<")[0] if "<" in i else i for i in edl]
    edl = [i.replace("n°", "").strip() for i in edl if i != '']
    return edl

In [ ]:
ed = edition(thesoup)
ed

#### 1.1.2 Edition Contents

In [ ]:
def contents(asoup):
    results = asoup.find(id="texto_pag_detalhe")
    results = results.text
    return results

In [ ]:
editioncontents = contents(thesoup)
print(editioncontents)

#### 1.1.3 Stories

In [ ]:
# All stories (individual)
def stories(asoup):
    storiesdict = {}
    # page section
    results = asoup.find(id="texto_pag_detalhe")
    # individual stories
    stories = results.find_all("div", class_="historia")
#     storieslist = [i.text for i in stories]
    for i in range(len(stories)):
        storiesdict[i] = stories[i].text
    return storiesdict

In [ ]:
stories = stories(thesoup)
stories

#### 1.1.4 Story Details

In [ ]:
def storydetails(contents, stories, storynumber):
    # return details only if story exists
    if (storynumber == 0 or storynumber > len(stories)):
        storydetails = None
    # for any story before the last
    elif storynumber < len(stories):
        storydetails = contents.split(stories[storynumber-1])[1].split(stories[storynumber])[0]
    # for the last story
    elif storynumber == len(stories):
        storydetails = contents.split(stories[storynumber-1])[1]
    return storydetails

In [ ]:
thestorydetail = storydetails(editioncontents, stories, 4)
print(thestorydetail)

#### 1.1.5 Main Character

In [ ]:
def maincharacter(astorydetail):
    character = astorydetail.split(",")[0].split("Personagens:")[1].strip()
    return character

In [ ]:
thecharacter = maincharacter(thestorydetail)
thecharacter

#### 1.1.6 Original Title

In [ ]:
def originaltitle(astorydetail):
    title = astorydetail.split("Publicada originalmente em")[1].split("(")[0].strip()
    return title

In [ ]:
theoriginal = originaltitle(thestorydetail)
theoriginal

#### 1.1.7 Original Number

In [ ]:
def originalnumber(astorydetail):
    issue = astorydetail.split("Publicada originalmente em")[1].split(")")[1].replace("n°\xa0", "").split("/")[0].strip()
    return issue

In [ ]:
thenumber = originalnumber(thestorydetail)
thenumber

#### 1.1.8 Original Year

In [ ]:
def originalyear(astorydetail):
    year = astorydetail.split("Publicada originalmente em")[1].split(")")[1].replace("n°\xa0", "").split("/")[1].split()[0].strip()
    return year

In [ ]:
theyear = originalyear(thestorydetail)
theyear

## 2 retrieve list of links for all editions

In [ ]:
def navigator(asoup):
    outlist = []
    nav = asoup.find(id="d0")
    for link in nav.find_all("a"):
        outlist.append(f'{link["href"].replace("../", "")}')
    return outlist

In [ ]:
alleditions = navigator(thesoup)

In [ ]:
alleditions

## 3. Save CSV and print log

In [ ]:
dict1 = {1:("a", "b"), 2:("c", "d")}

In [ ]:
dict1[2][0]

In [ ]:
list1 = [10, 9, 8]
list1.insert(0, 1)
list1

## 4. Test

In [1]:
from odinson import *

In [2]:
od = Odinson("http://www.guiadosquadrinhos.com/edicao/herois-da-tv-2-serie-n-1/htv0302/6274")

 Obtained Edition URL
 Retrieving page contents
 Retrieving all Editions URL list
 -------------------------------------------
  Heróis da TV #1
 -------------------------------------------


In [3]:
od.initialurl

'http://www.guiadosquadrinhos.com/edicao/herois-da-tv-2-serie-n-1/htv0302/6274'

In [4]:
od.editionurl

'edicao/herois-da-tv-2-serie-n-1/htv0302/6274'

In [5]:
od.editionnumber

['Heróis da TV', '1']

In [6]:
od.editioncontents[:800]

'\nCrédito da capa e editor\n\nDesenho:  John Buscema,  Keith Pollard Arte-Final:  Pablo Marcos Editor:  Helcio de Carvalho,  Cláudio Marra\n\nO Devorador de Planetas\nPersonagens:  Thor,  Galactus,  Tana Nile,  Odin,  Sif,  Registrador 211,  Ego, O Planeta Vivo\nRoteiro:  Stan Lee Desenho:  Jack Kirby Arte-Final:  Vince Colletta Cores:  Rita de Carvalho Letrista:  Natan Soares Editor original:  Stan Lee\nPublicada originalmente em   Thor (1966) n°\xa0160/1969 - Marvel Comics\r\n             \r\nTítulo original americano: And Now... Galactus!". Tana Nile pede ajuda ao deus do trovão para enfrentar o Devorador de Planetas, que ameaça inúmeros mundos.\r\n\r\n\r\n               Deve um deus prevalecer?\nPersonagens:  Thor,  Galactus,  Ego, O Planeta Vivo,  Registrador 211\nRoteiro:  Stan Lee Desenho:  Jack Kirby Art'

In [8]:
od.editionstories

{0: 'O Devorador de Planetas',
 1: 'Deve um deus prevalecer?',
 2: 'Duelo de ferro',
 3: 'Amargo adeus',
 4: 'Meia-Noite traz a morte!',
 5: 'A chama e a fúria'}

In [11]:
for i in od.editionstories.keys():
    if i < len(od.editionstories)-1:
        print(f"Chunk between {od.editionstories[i]} and {od.editionstories[i+1]}")
    else:
        print(f"Chunk after {od.editionstories[i]}")

Chunk between O Devorador de Planetas and Deve um deus prevalecer?
Chunk between Deve um deus prevalecer? and Duelo de ferro
Chunk between Duelo de ferro and Amargo adeus
Chunk between Amargo adeus and Meia-Noite traz a morte!
Chunk between Meia-Noite traz a morte! and A chama e a fúria
Chunk after A chama e a fúria


In [9]:
len(od.editionstories)

6